In [1]:
import pandas as pd
from skimage.io import imread
from sklearn.feature_extraction import image
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier

import timeit
import os

circle = imread("Data/Test/Circle/C10-1.png")
rectangle = imread("Data/Test/Rectangle/T10.png")
triangle = imread("Data/Test/Triangle/AT5845.png")
pentagone  = imread("Data/Test/Pentagon/P1.png")
hexagon = imread("Data/Test/Hexagon/H1.png")

circle_1 = image.extract_patches_2d(circle,(1,1))
triangle_1 = image.extract_patches_2d(triangle, (2, 2))
print "Circle",circle.shape,"circle_1",circle_1.shape
print "triangle",triangle.shape," triangle",triangle_1.shape
print "rectangle",rectangle.shape
print "pentagone",pentagone.shape
print "hexagon",hexagon.shape

Circle (28, 28, 4) circle_1 (784, 1, 1, 4)
triangle (28, 28, 3)  triangle (729, 2, 2, 3)
rectangle (28, 28, 3)
pentagone (28, 28, 3)
hexagon (28, 28, 3)


In [15]:
def process_image_test(directory):
    training_features = []
    training_labels = []
    for root, _, files in os.walk(directory):
            for file_name in files:
                file_path = os.path.join(root, file_name)
                im = imread(file_path)
                training_features.append(im)
                training_labels.append(file_path.split('/')[2])
    return training_features,training_labels

In [16]:
X,y = process_image_test('Data/Test/')

In [17]:
print len(X)
print len(y)

3857
3857


In [18]:
train_data = pd.DataFrame({'Image':X,'Name':y})

In [19]:
train_data.head()

,Image,Name
0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Rectangle
1,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Rectangle
2,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Rectangle
3,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Rectangle
4,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Rectangle


In [20]:
dummies = pd.get_dummies(train_data.Name)

In [21]:
train_data.Name = train_data.Name.map({"Circle":1,"Hexagon":2,"Pentagon":3,"Rectangle":4,"Triangle":5})

In [22]:
train_data.head(2)

,Image,Name
0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",4
1,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",4


In [24]:
X = train_data["Image"]
y = train_data["Name"]
print X[0].shape
print y[0]

(28, 28, 3)
4


In [25]:
knnClassifier = KNeighborsClassifier()

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [27]:
print X_train.shape
print y_train.shape

(3085,)
(3085,)


In [30]:
knnClassifier.fit(X,y)

ValueError: setting an array element with a sequence.